In [15]:
# --- Configuración de la Clase ---
import pandas as pd
import numpy as np

# Volvemos a cargar y limpiar el Titanic
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df_titanic = pd.read_csv(url)

# Rellenamos nulos (de la clase pasada) para que no molesten
media_edad = df_titanic['Age'].mean()
moda_embarked = df_titanic['Embarked'].mode()[0]
df_titanic['Age'].fillna(media_edad)
df_titanic['Embarked'].fillna(moda_embarked)

print("¡Dataset Titanic listo para transformar!")

¡Dataset Titanic listo para transformar!


In [16]:
# --- Ejemplo 1.1: .map() con un diccionario ---
# Queremos convertir la columna 'Sex' (texto) a números (0 o 1).
# Esto es vital para Machine Learning.

print(f"--- 1.1: Columna 'Sex' ANTES ---")
print(df_titanic['Sex'].head())

# Creamos una columna nueva 'Sex_Num'
df_titanic['Sex_Num'] = df_titanic['Sex'].map({'male': 0, 'female': 1})

print(f"\n--- Columna 'Sex_Num' DESPUÉS ---")
print(df_titanic[['Sex', 'Sex_Num']].head())


# --- Ejemplo 1.2: .replace() para correcciones ---
# .map() es estricto (reemplaza todo o pone NaN).
# .replace() es más flexible, solo reemplaza lo que le pides.
print(f"\n--- 1.2: Corregir 'Embarked' con .replace() ---")
print(f"Valores ANTES: {df_titanic['Embarked'].unique()}") # ['S', 'C', 'Q']

# Reemplacemos las letras por los nombres completos
df_titanic['Embarked'] = df_titanic['Embarked'].replace({
    'S': 'Southampton',
    'C': 'Cherbourg',
    'Q': 'Queenstown'
})
print(f"Valores DESPUÉS: {df_titanic['Embarked'].unique()}")

--- 1.1: Columna 'Sex' ANTES ---
0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

--- Columna 'Sex_Num' DESPUÉS ---
      Sex  Sex_Num
0    male        0
1  female        1
2  female        1
3  female        1
4    male        0

--- 1.2: Corregir 'Embarked' con .replace() ---
Valores ANTES: ['S' 'C' 'Q' nan]
Valores DESPUÉS: ['Southampton' 'Cherbourg' 'Queenstown' nan]


In [17]:
# --- Ejemplo 2.1: Funciones .str básicas ---
nombres = df_titanic['Name']

print(f"\n--- 2.1: Ejemplos de .str ---")
# Convertir a minúsculas
print(f"Minúsculas: {nombres.str.lower().head(1)}")

# Verificar si el texto contiene una palabra (devuelve True/False)
print(f"Contiene 'Mr.': {nombres.str.contains('Mr.').head(1)}")


# --- Ejemplo 2.2: Extracción de texto con .str.split() ---
# ¡Vamos a extraer el Título (Mr., Mrs., Miss.) del Nombre!
# El nombre es: "Braund, Mr. Owen Harris"

# 1. Separamos por la coma ',' -> ["Braund", " Mr. Owen Harris"]
#    y tomamos el segundo elemento [1]
paso_1 = df_titanic['Name'].str.split(',').str[1]
# print(paso_1.head()) # Output: " Mr. Owen Harris"

# 2. Separamos por el punto '.' -> [" Mr", " Owen Harris"]
#    y tomamos el primer elemento [0]
paso_2 = paso_1.str.split('.').str[0]
# print(paso_2.head()) # Output: " Mr"

# 3. Quitamos los espacios en blanco
paso_3 = paso_2.str.strip() # .strip() quita espacios
# print(paso_3.head()) # Output: "Mr"

# ¡Todo en una línea!
df_titanic['Title'] = df_titanic['Name'].str.split(',').str[1].str.split('.').str[0].str.strip()

print(f"\n--- Nueva columna 'Title' ---")
print(df_titanic[['Name', 'Title']].head())
print(f"\nTítulos encontrados: {df_titanic['Title'].unique()}")


--- 2.1: Ejemplos de .str ---
Minúsculas: 0    braund, mr. owen harris
Name: Name, dtype: object
Contiene 'Mr.': 0    True
Name: Name, dtype: bool

--- Nueva columna 'Title' ---
                                                Name Title
0                            Braund, Mr. Owen Harris    Mr
1  Cumings, Mrs. John Bradley (Florence Briggs Th...   Mrs
2                             Heikkinen, Miss. Laina  Miss
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)   Mrs
4                           Allen, Mr. William Henry    Mr

Títulos encontrados: ['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer']


In [18]:
# --- Ejemplo 3.1: .apply() en una Serie (con lambda) ---
# 'lambda' es una mini-función anónima
# Sintaxis: lambda variable_temporal: operación
# Ej: Cuál sería la edad al cuadrado

print(f"\n--- 3.1: .apply() en una Serie 'Age' ---")
print(df_titanic['Age'].apply(lambda x: x * x).head())


# --- Ejemplo 3.2: .apply(axis=1) en un DataFrame (EL GRAN PODER) ---
# Vamos a crear una columna 'IsAlone' (EstáSolo)
# Lógica: Si 'SibSp' (hermanos/esposa) + 'Parch' (padres/hijos) == 0, entonces True

# 1. Definimos una función que recibe una FILA
def check_is_alone(fila):
    # 'fila' es una Serie que representa la fila completa
    # Podemos acceder a sus columnas por nombre
    familiares = fila['SibSp'] + fila['Parch']
    if familiares == 0:
        return True
    else:
        return False

# 2. Usamos .apply() con axis=1 (para ir fila por fila)
df_titanic['IsAlone'] = df_titanic.apply(check_is_alone, axis=1)

print(f"\n--- 3.2: Nueva columna 'IsAlone' ---")
print(df_titanic[['SibSp', 'Parch', 'IsAlone']].head())


--- 3.1: .apply() en una Serie 'Age' ---
0     484.0
1    1444.0
2     676.0
3    1225.0
4    1225.0
Name: Age, dtype: float64

--- 3.2: Nueva columna 'IsAlone' ---
   SibSp  Parch  IsAlone
0      1      0    False
1      1      0    False
2      0      0     True
3      1      0    False
4      0      0     True


In [19]:
# --- Ejemplo 4.1: Agrupando 'Age' ---
# Vamos a "cortar" la edad en grupos
# (Usamos la 'Age' ya limpia)

# 1. Definimos los "bordes" de los contenedores (bins)
# 4 bordes = 3 grupos
bins = [0, 18, 65, 100]

# 2. Definimos las "etiquetas" para esos grupos
# 1 etiqueta menos que los bordes
labels = ['Menor de Edad', 'Adulto', 'Mayor']

# 3. Usamos pd.cut()
df_titanic['AgeGroup'] = pd.cut(
    df_titanic['Age'],
    bins=bins,
    labels=labels,
    right=True # right=True (default) significa que el borde derecho es inclusivo (0, 18]
)

print(f"\n--- 4.1: Nueva columna 'AgeGroup' ---")
print(df_titanic[['Age', 'AgeGroup']].head())

# Veamos el resultado
print(f"\nConteo de grupos:\n{df_titanic['AgeGroup'].value_counts()}")


--- 4.1: Nueva columna 'AgeGroup' ---
    Age AgeGroup
0  22.0   Adulto
1  38.0   Adulto
2  26.0   Adulto
3  35.0   Adulto
4  35.0   Adulto

Conteo de grupos:
AgeGroup
Adulto           567
Menor de Edad    139
Mayor              8
Name: count, dtype: int64
